# NGFC Modelling with IDEAS - pse
This is a modelling example of a NGFC system using idaes-pse framework.

In [1]:
from pyomo.environ import ConcreteModel, Constraint, Objective, SolverFactory, TransformationFactory, Constraint, Var
from pyomo.network import Arc

In [2]:
from idaes.core import FlowsheetBlock
from idaes.unit_models import Mixer, HeatExchanger, Separator, GibbsReactor, Heater
# Methane combustion ideal package got CH4, H2O, CO, CO2, N2, NH3, O2
import idaes.property_models.activity_coeff_models.methane_combustion_ideal as thermo_props
#import idaes.property_models.activity_coeff_models.methane_combustion_ideal as reaction_props

In [3]:
import matplotlib.pyplot as plt

In [4]:
from idaes.unit_models.separator import SplittingType
from idaes.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.core.util.model_statistics import degrees_of_freedom as dof

### Building Base Flowsheet.

In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.MethaneParameterBlock()

In [6]:
# Fuel ultilization (Uf): mole reductant consumed in FC per mole of reductant total
Uf = 0.8
# Air ultilization (Ua): mole of air consumed in FC per mole of air feed
Ua = 0.2
# Methane to steam ratio (MS): mole methane per mole water
MS = 2
# Electrical conversion efficiency (EE): 
EE = 0.9
#Methane LHV: (https://www.engineeringtoolbox.com/fuels-higher-calorific-values-d_169.html)
LHV = 50000*16.04 # (J/g * g/mol = J/mol) 
# Heat of water vaporization @ 25 C: (https://www.engineeringtoolbox.com/water-properties-d_1573.html)
enthalpy_vap = 43988 # (J/mol)
# Feed:
# Reaction: 
# Reforming: CH4 + H2O -> CO + 3H2
# Water gas shift: CO + H2O -> CO2 + H2
# Methane combustion: CH4 + 2O2 -> CO2 + 2H2O
# Hydrogen combustion: H2 + 1/2O2 -> H2O
# Carbon monoxide combustion: CO + 1/2O2 -> CO2

n_CH4f = 10
print("mole of methane feed: "+str(n_CH4f)+" mole/s")
n_H2Of = n_CH4f*MS
print("mole of steam feed: "+str(n_H2Of)+" mole/s")
n_O2f = n_CH4f*Uf*2/Ua
n_N2f = n_O2f*0.79/0.21
print("mole of air feed: "+str(n_N2f+n_O2f)+" mole/s")

n_H2ex = 2
n_COex = n_CH4f*(1-Uf)*4-n_H2ex
n_CO2ex = n_CH4f-n_COex
n_H2Oex = n_H2Of+2*n_CH4f-n_H2ex
y_H2ex = n_H2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_COex = n_COex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_CO2ex = n_CO2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_H2Oex = n_H2Oex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)

print("Anode exhaust: ")
print("y_H2ex: "+str(y_H2ex))
print("y_COex: "+str(y_COex))
print("y_CO2ex: "+str(y_CO2ex))
print("y_H2Oex: "+str(y_H2Oex))
print("Total mole/s: "+str(n_H2ex + n_COex + n_CO2ex + n_H2Oex))

n_N2ex = n_N2f
n_O2ex = n_O2f - n_CH4f*Uf*2
y_O2ex = n_O2ex/(n_O2ex+n_N2ex)
y_N2ex = n_N2ex/(n_O2ex+n_N2ex)
print("Cathode exhaust: ")
print("y_O2ex: "+str(y_O2ex))
print("y_N2ex: "+str(y_N2ex))
print("Total mole/s: "+str(n_O2ex+n_N2ex))

mole of methane feed: 10 mole/s
mole of steam feed: 20 mole/s
mole of air feed: 380.95238095238096 mole/s
Anode exhaust: 
y_H2ex: 0.04
y_COex: 0.11999999999999997
y_CO2ex: 0.08000000000000003
y_H2Oex: 0.76
Total mole/s: 50.0
Cathode exhaust: 
y_O2ex: 0.1753653444676409
y_N2ex: 0.824634655532359
Total mole/s: 364.95238095238096


In [7]:
# Temperature User input
T_FC_air_in = 700 + 273.15
T_FC_fuel_in = 500 + 273.15
T_FC_ex_out = 800 + 273.15

### Declare all Units:

In [8]:
m.fs.HX1c = Heater(default={"property_package": m.fs.thermo_params})

In [9]:
m.fs.HX2c = Heater(default={"property_package": m.fs.thermo_params})

In [10]:
m.fs.HX3c = Heater(default={"property_package": m.fs.thermo_params})

In [11]:
m.fs.HX1h = Heater(default={"property_package": m.fs.thermo_params})

In [12]:
m.fs.HX2h = Heater(default={"property_package": m.fs.thermo_params})

In [13]:
m.fs.HX3h = Heater(default={"property_package": m.fs.thermo_params})

In [14]:
m.fs.Boiler = Heater(default={"property_package": m.fs.thermo_params})

In [15]:
m.fs.Mix1 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [16]:
m.fs.Mix2 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [17]:
m.fs.Mix3 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [18]:
m.fs.Mix4 = Mixer(default={"dynamic": False,
                           "num_inlets": 3,
                           "property_package": m.fs.thermo_params})

In [19]:
m.fs.Split1 = Separator(default={"dynamic": False,
                                 "split_basis": SplittingType.componentFlow,
                                 "property_package": m.fs.thermo_params})

In [20]:
m.fs.Split2 = Separator(default={"dynamic": False,
                                 "num_outlets": 3,
                                 "split_basis": SplittingType.totalFlow,
                                 "property_package": m.fs.thermo_params})

In [21]:
m.fs.Reformer = GibbsReactor(default={"dynamic": False,
                                      "property_package": m.fs.thermo_params,
                                      "has_pressure_change": False,
                                      "has_heat_transfer": True})

In [22]:
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})

In [23]:
m.fs.Burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### Declare all Streams:

In [24]:
m.fs.stream0 = Arc(source=m.fs.Mix1.outlet,
                   destination=m.fs.HX1c.inlet)

In [25]:
m.fs.stream1 = Arc(source=m.fs.Split1.outlet_1,
                   destination=m.fs.HX3c.inlet)

In [26]:
m.fs.stream2 = Arc(source=m.fs.HX1c.outlet,
                   destination=m.fs.Reformer.inlet)

In [27]:
m.fs.stream3 = Arc(source=m.fs.Split1.outlet_2,
                   destination=m.fs.HX2c.inlet)

In [28]:
m.fs.stream4 = Arc(source=m.fs.Reformer.outlet,
                   destination=m.fs.Mix2.inlet_1)

In [29]:
m.fs.stream5 = Arc(source=m.fs.HX3c.outlet,
                   destination=m.fs.Mix2.inlet_2)

In [30]:
m.fs.stream6 = Arc(source=m.fs.Mix2.outlet,
                   destination=m.fs.SOFC.inlet)

In [31]:
m.fs.stream7 = Arc(source=m.fs.HX2c.outlet,
                   destination=m.fs.Mix3.inlet_2)

In [32]:
m.fs.stream8 = Arc(source=m.fs.SOFC.outlet,
                   destination=m.fs.Mix3.inlet_1)

In [33]:
m.fs.stream9 = Arc(source=m.fs.Mix3.outlet,
                   destination=m.fs.Burner.inlet)

In [34]:
m.fs.stream10 = Arc(source=m.fs.Burner.outlet,
                    destination=m.fs.Split2.inlet)

In [35]:
m.fs.stream11 = Arc(source=m.fs.Split2.outlet_1,
                    destination=m.fs.HX1h.inlet)

In [36]:
m.fs.stream12 = Arc(source=m.fs.Split2.outlet_2,
                    destination=m.fs.HX2h.inlet)

In [37]:
m.fs.stream13 = Arc(source=m.fs.Split2.outlet_3,
                    destination=m.fs.HX3h.inlet)

In [38]:
m.fs.stream14 = Arc(source=m.fs.HX1h.outlet,
                    destination=m.fs.Mix4.inlet_1)

In [39]:
m.fs.stream15 = Arc(source=m.fs.HX2h.outlet,
                    destination=m.fs.Boiler.inlet)

In [40]:
m.fs.stream17 = Arc(source=m.fs.Boiler.outlet,
                    destination=m.fs.Mix4.inlet_2)

In [41]:
m.fs.stream16 = Arc(source=m.fs.HX3h.outlet,
                    destination=m.fs.Mix4.inlet_3)

In [42]:
TransformationFactory("network.expand_arcs").apply_to(m)

### Define known Material Streams:

In [43]:
# Fix methane flow to Mix1:
m.fs.Mix1.inlet_1.flow_mol.fix(n_CH4f)
m.fs.Mix1.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Mix1.inlet_1.mole_frac_comp[0.0,"CH4"].fix(1.0)
m.fs.Mix1.inlet_1.temperature.fix(25+273.15)
m.fs.Mix1.inlet_1.pressure.fix(101325)

In [44]:
# Fix water flow to Mix1:
m.fs.Mix1.inlet_2.flow_mol.fix(n_H2Of)
m.fs.Mix1.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Mix1.inlet_2.mole_frac_comp[0.0,"H2O"].fix(1.0)
m.fs.Mix1.inlet_2.temperature.fix(25+273.15)
m.fs.Mix1.inlet_2.pressure.fix(101325)

In [45]:
# Fix air flow to Split1:
m.fs.Split1.inlet.flow_mol.fix(n_N2f+n_O2f)
m.fs.Split1.inlet.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Split1.inlet.mole_frac_comp[0.0,"O2"].fix(0.21)
m.fs.Split1.inlet.mole_frac_comp[0.0,"N2"].fix(0.79)
m.fs.Split1.inlet.temperature.fix(25+273.15)
m.fs.Split1.inlet.pressure.fix(101325)

In [46]:
# Fix O2 flow in Split1 outlet_1:
m.fs.Split1.outlet_1.flow_mol.fix(n_CH4f*Uf*2)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CH4"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CO"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CO2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"H2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"H2O"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"N2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"O2"].fix(1.0)

### Constraints:  Adiabatic Burner

In [47]:
m.fs.Burner.heat_duty.fix(0.0)

In [48]:
m.fs.Reformer.heat_duty.fix(0.0)

### Constraints: Reformer temperature

In [49]:
m.fs.Reformer.outlet.temperature.fix(T_FC_fuel_in)

### Constraints: SOFC exit temperature

In [50]:
m.fs.SOFC.outlet.temperature.fix(T_FC_ex_out)

### Constraints: Heat Exchanger tube outlet temperature

In [51]:
m.fs.HX2c.outlet.temperature.fix(T_FC_ex_out)

In [52]:
m.fs.HX3c.outlet.temperature.fix(T_FC_air_in)

In [53]:
m.fs.HX1_energy_balance = Constraint(expr = m.fs.HX1c.heat_duty[0] + m.fs.HX1h.heat_duty[0] ==0 )

In [54]:
m.fs.HX2_energy_balance = Constraint(expr = m.fs.HX2c.heat_duty[0] + m.fs.HX2h.heat_duty[0] ==0 )

In [55]:
m.fs.HX3_energy_balance = Constraint(expr = m.fs.HX3c.heat_duty[0] + m.fs.HX3h.heat_duty[0] ==0 )

In [56]:
m.fs.Boiler.heat_duty.fix(-n_H2Of*enthalpy_vap)

In [57]:
m.fs.Split2.split_fraction[0.0, "outlet_1"].fix(0.09)

In [58]:
m.fs.Split2.split_fraction[0.0, "outlet_3"].fix(0.03)

In [ ]:
m.fs.HX3h.outlet.temperature.fix(100+273.15)

In [ ]:
m.fs.HX1h.outlet.temperature.fix(100+273.15)

In [59]:
dof(m)

0

### Initialize Stuff

In [60]:
m.fs.Mix1.initialize()
m.fs.Mix2.initialize()
m.fs.Mix3.initialize()
m.fs.Mix4.initialize()
m.fs.HX1c.initialize()
m.fs.HX2c.initialize()
m.fs.HX3c.initialize()
m.fs.HX1h.initialize()
m.fs.HX2h.initialize()
m.fs.HX3h.initialize()
m.fs.Boiler.initialize()
m.fs.Split1.initialize()
m.fs.Split2.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-29 13:05:26 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.mixed_state State Released.
2020-04-29 13:05:26 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_1_state State Released.
2020-04-29 13:05:26 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_2_state State Released.


--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

...

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-29 13:05:32 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Boiler.control_volume.properties_out State Released.
    model=fs.Boiler;
        message from solver=Ipopt 3.13.2\x3a Converged to a locally infeasible
        point. Problem may be infeasible.
2020-04-29 13:05:32 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Boiler.control_volume.properties_in State Released.


--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-29 13:05:32 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Split1.outlet_1_state State Released.


KeyError: 'fs.Split1 state variable and split fraction indexing sets do not match. The in-built  initialization routine for Separators relies on the split fraction and state variable indexing sets matching to calculate initial guesses for extensive variables. In other cases users will need to provide their own initial guesses'

In [61]:
solver = SolverFactory('ipopt')

In [62]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3965
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1045

Total number of variables............................:     1145
                     variables with only lower bounds:      106
                variables with lower and upper bounds:      609
                     variables with only upper bounds:        0
Total

 185r 0.0000000e+00 4.21e+05 4.62e+03  -2.7 4.28e+02  -0.3 1.00e+00 1.00e+00f  1
 186r 0.0000000e+00 4.20e+05 8.27e+03  -2.7 1.16e+03  -0.8 9.73e-01 8.05e-01f  1
 187r 0.0000000e+00 4.20e+05 8.24e+03  -2.7 4.26e+02  -0.3 1.00e+00 4.50e-03f  1
 188r 0.0000000e+00 4.20e+05 8.23e+03  -2.7 2.07e+01   1.0 1.00e+00 1.95e-04f  1
 189r 0.0000000e+00 4.20e+05 6.79e+03  -2.7 6.19e+01   0.5 1.00e+00 1.76e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 4.20e+05 4.17e+03  -2.7 1.84e+02   0.0 1.00e+00 3.91e-01f  1
 191r 0.0000000e+00 4.20e+05 4.16e+03  -2.7 6.91e+01   0.5 1.00e+00 2.13e-03f  1
 192r 0.0000000e+00 4.20e+05 4.16e+03  -2.7 2.05e+02  -0.0 1.00e+00 2.89e-04f  1
 193r 0.0000000e+00 4.20e+05 2.06e+02  -2.7 7.77e+01   0.4 1.00e+00 1.00e+00f  1
 194r 0.0000000e+00 4.20e+05 2.06e+02  -2.7 2.91e+01   0.9 1.00e+00 1.00e+00f  1
 195r 0.0000000e+00 4.20e+05 2.06e+02  -2.7 1.09e+01   1.3 3.46e-01 2.34e-01f  1
 196r 0.0000000e+00 4.20e+05

 369r 0.0000000e+00 3.43e+05 1.31e+02  -4.0 9.79e+02  -1.0 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 0.0000000e+00 3.43e+05 8.62e+01  -4.0 3.61e+02  -0.6 1.00e+00 1.00e+00f  1
 371r 0.0000000e+00 3.43e+05 1.41e+02  -4.0 1.03e+03  -1.1 1.00e+00 1.00e+00f  1
 372r 0.0000000e+00 3.42e+05 8.06e+01  -4.0 3.80e+02  -0.7 1.00e+00 1.00e+00f  1
 373r 0.0000000e+00 3.42e+05 1.48e+02  -4.0 1.08e+03  -1.2 1.00e+00 1.00e+00f  1
 374r 0.0000000e+00 3.41e+05 7.47e+01  -4.0 3.96e+02  -0.7 1.00e+00 1.00e+00f  1
 375r 0.0000000e+00 3.41e+05 1.52e+02  -4.0 1.12e+03  -1.2 1.00e+00 1.00e+00f  1
 376r 0.0000000e+00 3.40e+05 6.87e+01  -4.0 4.10e+02  -0.8 1.00e+00 1.00e+00f  1
 377r 0.0000000e+00 3.39e+05 1.53e+02  -4.0 1.15e+03  -1.3 1.00e+00 1.00e+00f  1
 378r 0.0000000e+00 3.39e+05 6.26e+01  -4.0 4.20e+02  -0.8 1.00e+00 1.00e+00f  1
 379r 0.0000000e+00 3.38e+05 1.49e+02  -4.0 1.17e+03  -1.3 1.00e+00 1.00e+00f  1
iter    objective    inf_pr 

 552r 0.0000000e+00 3.31e+05 2.80e+00  -4.0 1.62e+01  -0.8 1.00e+00 1.00e+00f  1
 553r 0.0000000e+00 3.31e+05 2.80e+00  -4.0 4.85e+01  -1.2 1.00e+00 1.00e+00f  1
 554r 0.0000000e+00 3.31e+05 2.80e+00  -4.0 1.82e+01  -0.8 1.00e+00 1.00e+00f  1
 555r 0.0000000e+00 3.31e+05 2.80e+00  -4.0 5.45e+01  -1.3 1.00e+00 1.00e+00f  1
 556r 0.0000000e+00 3.31e+05 2.80e+00  -4.0 2.04e+01  -0.9 1.00e+00 1.00e+00f  1
 557r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 6.13e+01  -1.3 1.00e+00 1.00e+00f  1
 558r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 2.30e+01  -0.9 1.00e+00 1.00e+00f  1
 559r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 8.62e+00  -0.5 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 2.58e+01  -1.0 1.00e+00 1.00e+00f  1
 561r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 9.69e+00  -0.5 1.00e+00 1.00e+00f  1
 562r 0.0000000e+00 3.31e+05 2.79e+00  -4.0 2.91e+01  -1.0 1.00e+00 1.00e+00f  1
 563r 0.0000000e+00 3.31e+05

 737r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 9.00e+00  -0.5 1.00e+00 1.00e+00f  1
 738r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 2.70e+01  -1.0 1.00e+00 1.00e+00f  1
 739r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 1.01e+01  -0.6 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 740r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 3.04e+01  -1.1 1.00e+00 1.00e+00f  1
 741r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 1.14e+01  -0.6 1.00e+00 1.00e+00f  1
 742r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 3.41e+01  -1.1 1.00e+00 1.00e+00f  1
 743r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 1.28e+01  -0.7 1.00e+00 1.00e+00f  1
 744r 0.0000000e+00 3.31e+05 2.68e+00  -4.0 3.84e+01  -1.2 1.00e+00 1.00e+00f  1
 745r 0.0000000e+00 3.31e+05 2.67e+00  -4.0 1.44e+01  -0.7 1.00e+00 1.00e+00f  1
 746r 0.0000000e+00 3.31e+05 2.67e+00  -4.0 4.32e+01  -1.2 1.00e+00 1.00e+00f  1
 747r 0.0000000e+00 3.31e+05 2.67e+00  -4.0 1.62e+01  -0.8 1.00e+00 1.00e+00f  1
 748r 0.0000000e+00 3.31e+05

...

2208r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 4.00e+00  -0.7 1.00e+00 1.00e+00f  1
2209r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 1.20e+01  -1.1 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2210r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 4.50e+00  -0.7 1.00e+00 1.00e+00f  1
2211r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 1.35e+01  -1.2 1.00e+00 1.00e+00f  1
2212r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 5.06e+00  -0.8 1.00e+00 1.00e+00f  1
2213r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 1.52e+01  -1.3 1.00e+00 1.00e+00f  1
2214r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 5.69e+00  -0.8 1.00e+00 1.00e+00f  1
2215r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 2.13e+00  -0.4 1.00e+00 1.00e+00f  1
2216r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 6.40e+00  -0.9 1.00e+00 1.00e+00f  1
2217r 0.0000000e+00 3.31e+05 8.53e-01  -4.0 2.40e+00  -0.4 1.00e+00 1.00e+00f  1
2218r 0.0000000e+00 3.31e+05 8.52e-01  -4.0 7.20e+00  -0.9 1.00e+00 1.00e+00f  1
2219r 0.0000000e+00 3.31e+05

2391r 0.0000000e+00 3.31e+05 8.43e-01  -4.0 1.63e+01  -1.3 1.00e+00 1.00e+00f  1
2392r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 6.12e+00  -0.9 1.00e+00 1.00e+00f  1
2393r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 2.30e+00  -0.4 1.00e+00 1.00e+00f  1
2394r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 6.89e+00  -0.9 1.00e+00 1.00e+00f  1
2395r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 2.58e+00  -0.5 1.00e+00 1.00e+00f  1
2396r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 7.75e+00  -1.0 1.00e+00 1.00e+00f  1
2397r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 2.91e+00  -0.5 1.00e+00 1.00e+00f  1
2398r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 8.71e+00  -1.0 1.00e+00 1.00e+00f  1
2399r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 3.27e+00  -0.6 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2400r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 9.80e+00  -1.1 1.00e+00 1.00e+00f  1
2401r 0.0000000e+00 3.31e+05 8.42e-01  -4.0 3.68e+00  -0.6 1.00e+00 1.00e+00f  1
2402r 0.0000000e+00 3.31e+05

2575r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 3.13e+00  -0.6 1.00e+00 1.00e+00f  1
2576r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 9.38e+00  -1.1 1.00e+00 1.00e+00f  1
2577r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 3.52e+00  -0.6 1.00e+00 1.00e+00f  1
2578r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 1.06e+01  -1.1 1.00e+00 1.00e+00f  1
2579r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 3.96e+00  -0.7 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2580r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 1.19e+01  -1.2 1.00e+00 1.00e+00f  1
2581r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 4.45e+00  -0.7 1.00e+00 1.00e+00f  1
2582r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 1.34e+01  -1.2 1.00e+00 1.00e+00f  1
2583r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 5.01e+00  -0.8 1.00e+00 1.00e+00f  1
2584r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 1.50e+01  -1.3 1.00e+00 1.00e+00f  1
2585r 0.0000000e+00 3.31e+05 8.32e-01  -4.0 5.63e+00  -0.8 1.00e+00 1.00e+00f  1
2586r 0.0000000e+00 3.31e+05

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2760r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 4.48e+00  -1.2 1.00e+00 1.00e+00f  1
2761r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 1.68e+00  -0.8 1.00e+00 1.00e+00f  1
2762r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 6.30e-01  -0.4 1.00e+00 1.00e+00f  1
2763r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 1.89e+00  -0.9 1.00e+00 1.00e+00f  1
2764r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 7.09e-01  -0.4 1.00e+00 1.00e+00f  1
2765r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.13e+00  -0.9 1.00e+00 1.00e+00f  1
2766r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 7.97e-01  -0.5 1.00e+00 1.00e+00f  1
2767r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.39e+00  -1.0 1.00e+00 1.00e+00f  1
2768r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 8.97e-01  -0.5 1.00e+00 1.00e+00f  1
2769r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.69e+00  -1.0 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2770r 0.0000000e+00 3.31e+05

2943r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.31e+00  -1.0 1.00e+00 1.00e+00f  1
2944r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 8.68e-01  -0.5 1.00e+00 1.00e+00f  1
2945r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.60e+00  -1.0 1.00e+00 1.00e+00f  1
2946r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 9.76e-01  -0.6 1.00e+00 1.00e+00f  1
2947r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 2.93e+00  -1.1 1.00e+00 1.00e+00f  1
2948r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 1.10e+00  -0.6 1.00e+00 1.00e+00f  1
2949r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 3.29e+00  -1.1 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2950r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 1.24e+00  -0.7 1.00e+00 1.00e+00f  1
2951r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 3.71e+00  -1.2 1.00e+00 1.00e+00f  1
2952r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 1.39e+00  -0.7 1.00e+00 1.00e+00f  1
2953r 0.0000000e+00 3.31e+05 2.56e-01  -4.0 4.17e+00  -1.2 1.00e+00 1.00e+00f  1
2954r 0.0000000e+00 3.31e+05

In [ ]:
m.fs.Burner.outlet.display()

In [ ]:
m.fs.Reformer.inlet.display()

In [ ]:
m.fs.HX2.display()

In [ ]:
m.fs.Mix4.inlet_1.display()

In [ ]:
m.fs.Mix4.inlet_2.display()

In [ ]:
m.fs.Mix4.inlet_3.display()

In [ ]:
m.fs.Burner.outlet.display()

In [ ]:
m.fs.SOFC.heat_duty.display()

In [ ]:
m.fs.Split2.split_fraction.display()

In [ ]:
m.fs.HX1.inlet_1.display()

In [ ]:
m.fs.HX1.tube_inlet.display()

In [ ]:
m.fs.Mix1.inlet_1.display()

In [ ]:
m.fs.Mix1.inlet_2.display()

In [ ]:
m.fs.Mix1.outlet.display()

In [ ]:
m.fs.Heater.display()

In [ ]:
m.fs.Mix4.inlet_1.display()

In [ ]:
m.fs.Mix4.inlet_2.display()

In [ ]:
m.fs.Mix4.inlet_3.display()

In [ ]:
m.fs.Mix4.outlet.display()

In [ ]:
354.9024586155912-273.15

### First mixer (Mix1) combines methane and water stream:

In [ ]:
m.fs.mix1 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [ ]:
m.fs.mix1.inlet_1.flow_mol.fix(n_CH4f)
m.fs.mix1.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix1.inlet_1.mole_frac_comp[0.0,"CH4"].fix(1.0)
m.fs.mix1.inlet_1.temperature.fix(25+273.15)
m.fs.mix1.inlet_1.pressure.fix(101325)

In [ ]:
m.fs.mix1.inlet_2.flow_mol.fix(n_H2Of)
m.fs.mix1.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix1.inlet_2.mole_frac_comp[0.0,"H2O"].fix(1.0)
m.fs.mix1.inlet_2.temperature.fix(25+273.15)
m.fs.mix1.inlet_2.pressure.fix(101325)

In [ ]:
m.fs.mix1.inlet_1.display()

In [ ]:
m.fs.mix1.inlet_2.display()

### First heat exchanger (HX1) heats up methane & water stream
### Also reformer (Reformer) is a Gibbs Reactor that receive heat from HX1 exhaust stream

In [ ]:
# Set up reformer?
m.fs.reformer = GibbsReactor(default={"dynamic": False,
                                      "property_package": m.fs.thermo_params,
                                      "has_pressure_change": False,
                                      "has_heat_transfer": True})

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix1.outlet,
                   destination=m.fs.reformer.inlet)

In [ ]:
m.fs.reformer.outlet.temperature.fix(T_FC_fuel_in+273.15)

### HX1: Provide the heat duty for Reformer

### Seperator (Sep1) split the air into a single of stream of O2 going to SOFC

### Mix2: combined reformer outlet stream and O2 stream from cathode.

In [ ]:
m.fs.mix2 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [ ]:
m.fs.stream2 = Arc(source=m.fs.reformer.outlet,
                   destination=m.fs.mix2.inlet_1)

In [ ]:
m.fs.mix2.inlet_2.flow_mol.fix(n_CH4f*Uf*2)
m.fs.mix2.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix2.inlet_2.mole_frac_comp[0.0,"O2"].fix(1.0)
m.fs.mix2.inlet_2.temperature.fix(T_FC_air_in+273.15)
m.fs.mix2.inlet_2.pressure.fix(101325)

### SOFC: Gibbs Reactor w stream coming from mix2

In [ ]:
# Set up SOFC
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})

In [ ]:
m.fs.stream3 = Arc(source=m.fs.mix2.outlet,
                   destination=m.fs.SOFC.inlet)

In [ ]:
m.fs.SOFC.outlet.temperature.fix(T_FC_ex_out+273.15)

In [ ]:
m.fs.mix1.initialize()

In [ ]:
m.fs.mix2.initialize()

### SOFC outlet come into the burner with the rest of the air
### Burner: Gibbs Reactor

In [ ]:
# Feed to the burner

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
m.fs.reformer.inlet.display()

In [ ]:
m.fs.reformer.outlet.display()

In [ ]:
m.fs.reformer.heat_duty.display()

In [ ]:
m.fs.mix2.inlet_1.display()

In [ ]:
m.fs.mix2.inlet_2.display()

In [ ]:
m.fs.mix2.outlet.display()

In [ ]:
m.fs.SOFC.inlet.display()

In [ ]:
m.fs.SOFC.outlet.display()

In [ ]:
m.fs.SOFC.heat_duty.display()

In [ ]:
m.fs.burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### SOFC: Gibbs Reactor, taking Reformer outlet and a specified O2 amount from Mix2

### Mixing anode & cathode exhaust:

In [ ]:
m.fs.mix_exhaust = Mixer(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params})

In [ ]:
# First inlet to mix_exhaust : anode exhaust
m.fs.mix_exhaust.inlet_1.flow_mol.fix(50.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2"].fix(0.04)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO"].fix(0.12)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO2"].fix(0.08)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2O"].fix(0.76)
m.fs.mix_exhaust.inlet_1.temperature.fix(T_FC_ex_out+273.15)
m.fs.mix_exhaust.inlet_1.pressure.fix(101325)

In [ ]:
m.fs.mix_exhaust.inlet_1.display()

In [ ]:
# Second inlet to mix_exhaust : cathode exhaust
m.fs.mix_exhaust.inlet_2.flow_mol.fix(364.95238095238096)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"O2"].fix(0.1753653444676409)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"N2"].fix(0.824634655532359)
m.fs.mix_exhaust.inlet_2.temperature.fix(T_FC_ex_out+273.15)
m.fs.mix_exhaust.inlet_2.pressure.fix(101325)

In [ ]:
m.fs.mix_exhaust.inlet_2.display()

### Burner as Gibbs Reactor

In [ ]:
m.fs.burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### Point mixer stream to Burner:

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix_exhaust.outlet,
                  destination=m.fs.burner.inlet)

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

In [ ]:
m.fs.burner.heat_duty.fix(0.0)

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

In [ ]:
solver = SolverFactory('ipopt')

In [ ]:
results = solver.solve(m, tee=True)

In [ ]:
print(results)

In [ ]:
m.fs.burner.inlet.display()

In [ ]:
m.fs.burner.outlet.display()

### Make another Gibbs Reactor to represent the SOFC

In [ ]:
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})